# Stats from inside the Oxford Ring Road

A coarse look at all the STATS19 data inside the Oxford Ring Road, 1999-2000. 

Data from https://bikedata.cyclestreets.net/ and can be found in `data/inside_ring_road`. It had to be downloaded one year at a time due to the 400 data point limit.

<div>
<img src="../data/inside_ring_road/Screenshot%202022-08-06%20at%2020.41.31.png" width="500"/>
</div>

In [ ]:
%pip install tabulate

  Using cached tabulate-0.8.10-py3-none-any.whl (29 kB)

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import datetime 
import pathlib

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
data_dir = pathlib.Path.cwd().parent.joinpath("data/inside_ring_road")

In [5]:
# Read in each of the .csv files into a pandas dataframe and concatenate into a single dataframe
df = pd.concat([pd.read_csv(f) for f in data_dir.glob("*.csv")], ignore_index=True)


In [6]:
df.head()

,Accident Index,Timestamp,severity,Casualties,Number of Casualties,Number of Vehicles,url,apiUrl,Latitude,Longitude
0,200143P067101,"11:40am, 5th October 2001",fatal,Pedestrian,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.759808,-1.212196
1,200143P296091,"2:47am, 27th September 2001",fatal,Pedestrian,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.745090,-1.229978
2,200143P069111,"5:24pm, 6th November 2001",serious,Pedestrian,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.744275,-1.229123
3,200143P130041,"10:50am, 10th April 2001",serious,Cyclist,1,2,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.750375,-1.240753
4,200143P143011,"5:15pm, 12th January 2001",serious,Pedestrian,1,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.767750,-1.230172


In [15]:
# Get simple stats by severity and type of casualty 
df[['Accident Index', 'severity', 'Casualties']].groupby(['severity', 'Casualties']).count()

Accident Index
severity Casualties                                           
fatal    Cyclist                                             9
         Pedestrian                                         20
         Pedestrian,Car occupant                             1
serious  Cyclist                                           453
         Cyclist,Bus/coach/minibus occupant                  4
         Cyclist,Car occupant                                1
         Cyclist,Car occupant,Other                          1
         Cyclist,Motorcycle rider/passenger                  2
         Cyclist,Taxi occupant                               1
         Pedestrian                                        284
         Pedestrian,Bus/coach/minibus occupant               5
         Pedestrian,Car occupant                             4
         Pedestrian,Cyclist                                  9
         Pedestrian,Motorcycle rider/passenger               3
slight   Cyclist                                          2703
         Cyclist,Bus/coach/minibus occupant                  5
         Cyclist,Car occupant                               12
         Cyclist,Goods vehicle occupant                      1
         Cyclist,Motorcycle rider/passenger                  5
         Cyclist,Taxi occupant                               1
         Pedestrian                                       1036
         Pedestrian,Bus/coach/minibus occupant               7
         Pedestrian,Car occupant                             8
         Pedestrian,Cyclist                                 13
         Pedestrian,Motorcycle rider/passenger              10
         Pedestrian,Taxi occupant                            1

In [91]:
counts = df[['Accident Index', 'severity', 'Casualties']].groupby(['severity', 'Casualties'])

In [93]:
df.loc[(df['Casualties'] == 'Pedestrian,Car occupant') & (df['severity'] == 'fatal')]

,Accident Index,Timestamp,severity,Casualties,Number of Casualties,Number of Vehicles,url,apiUrl,Latitude,Longitude
1234,201143P273061,"1:48am, 25th June 2011",fatal,"Pedestrian,Car occupant",3,1,https://www.cyclestreets.net/collisions/report...,https://api.cyclestreets.net/v2/collisions.loc...,51.728645,-1.259361


In [94]:
casualty_types = df['Casualties'].unique()
casualty_types_cyclist = [x for x in casualty_types if 'Cyclist' in x]
casualty_types_cyclist.remove('Pedestrian,Cyclist')  # assume more serious for pedestrian here
casualty_types_pedestrian = [x for x in casualty_types if 'Pedestrian' in x]

In [96]:
casualty_types_cyclist

['Cyclist',
 'Cyclist,Motorcycle rider/passenger',
 'Cyclist,Car occupant',
 'Cyclist,Bus/coach/minibus occupant',
 'Cyclist,Taxi occupant',
 'Cyclist,Car occupant,Other',
 'Cyclist,Goods vehicle occupant']

In [65]:
casualty_types_pedestrian

['Pedestrian',
 'Pedestrian,Cyclist',
 'Pedestrian,Bus/coach/minibus occupant',
 'Pedestrian,Motorcycle rider/passenger',
 'Pedestrian,Car occupant',
 'Pedestrian,Taxi occupant']

In [97]:
counts = df[['Accident Index', 'severity', 'Casualties']].groupby(['severity', 'Casualties'])

In [98]:
def map_none_to_zero(x):
    if x is None:
        return 0
    else:
        return len(x)

In [99]:
def get_info_from_counts(counts, tuple):
    try: 
        return map_none_to_zero(counts.get_group(tuple))
    except KeyError:
        return 0

In [100]:
sum(get_info_from_counts(counts, ('fatal', c)) for c in casualty_types_pedestrian)

21

In [101]:
# Hand cranking a bit of this as haven't quite worked out the API thing yet 
counts = df[['Accident Index', 'severity', 'Casualties']].groupby(['severity', 'Casualties'])

casualty_counts = {
    'fatal': {
        'Pedestrian': sum(get_info_from_counts(counts, ('fatal', c)) for c in casualty_types_pedestrian),
        'Cyclist': sum(get_info_from_counts(counts, ('fatal', c)) for c in casualty_types_cyclist),
    },
    'serious':{
        'Pedestrian': sum(get_info_from_counts(counts, ('serious', c)) for c in casualty_types_pedestrian),
        'Cyclist': sum(get_info_from_counts(counts, ('serious', c)) for c in casualty_types_cyclist),
    },
    'slight': {
        'Pedestrian': sum(get_info_from_counts(counts, ('slight', c)) for c in [i for i in casualty_types_pedestrian if i != 'Pedestrian,Cyclist']),
        'Cyclist': sum(get_info_from_counts(counts, ('slight', c)) for c in casualty_types_cyclist + ['Pedestrian,Cyclist']), 
    }
}

In [115]:
# Quick check
total = 0
for severity in casualty_counts:
    for casualty_type in casualty_counts[severity]:
        total += casualty_counts[severity][casualty_type]

total

4599

In [108]:
len(df)

4599

In [114]:
df = pd.DataFrame(
    data=casualty_counts
)
print(df.to_markdown())

|            |   fatal |   serious |   slight |
|:-----------|--------:|----------:|---------:|
| Pedestrian |      21 |       305 |     1062 |
| Cyclist    |       9 |       462 |     2740 |
